# Demonstration of the Temperature Problem

### We'll start off by importing what we need

In [1]:
# The usual suspects...
import numpy as np
import numpy.linalg as npla

# We'll need to import these to deal with SPARSE matrices
import scipy
import scipy.sparse.linalg as spla
from scipy import sparse
from scipy import linalg

# These are so we can make pretty 2D & 3D plots...
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
%matplotlib tk

## We'll create 3 functions for our use:
### 1. make_A(k)    --- creates the nxn matrix we need from a kxk sized room model
### 2. make_b(k)    --- creates the right-hand side vector in the Ax = b setup
### 3. radiator(k)    --- creates a radiator wall at a set temperature (default is 100)

In [2]:
def make_A(k):
    """Create the matrix for the temperature problem on a k-by-k grid (2-dimensional).
    Parameters: 
      k: number of grid points in each dimension.
    Outputs:
      A: the sparse k**2-by-k**2 matrix representing the finite difference approximation to Poisson's equation.
    """
    # First make a list with one triple (row, column, value) for each nonzero element of A
    triples = []
    for i in range(k):
        for j in range(k):
            # what row of the matrix is grid point (i,j)?
            row = j + i*k
            # the diagonal element in this row
            triples.append((row, row, 4.0))
            # connect to left grid neighbor
            if j > 0:
                triples.append((row, row - 1, -1.0))
            # ... right neighbor
            if j < k - 1:
                triples.append((row, row + 1, -1.0))
            # ... neighbor above
            if i > 0:
                triples.append((row, row - k, -1.0))
            # ... neighbor below
            if i < k - 1:
                triples.append((row, row + k, -1.0))
    
    # Finally convert the list of triples to a scipy sparse matrix
    ndim = k*k
    rownum = [t[0] for t in triples]
    colnum = [t[1] for t in triples]
    values = [t[2] for t in triples]
    A = sparse.csr_matrix((values, (rownum, colnum)), shape = (ndim, ndim))
    
    return A 

In [23]:
def make_b(k, top = 0, bottom = 0, left = 0, right = 0):
    """Create the right-hand side for the temperature problem on a k-by-k grid.
    Parameters: 
      k: number of grid points in each dimension.
      top: list of k values for top boundary (optional, defaults to 0)
      bottom: list of k values for bottom boundary (optional, defaults to 0)
      left: list of k values for top boundary (optional, defaults to 0)
      right: list of k values for top boundary (optional, defaults to 0)
    Outputs:
      b: the k**2 element vector (as a numpy array) for the rhs of the Poisson equation with given boundary conditions
    """
    # Start with a vector of zeros
    ndim = k*k
    b = np.zeros(shape = (ndim,ndim))
    print(b)
    # Fill in the four boundaries as appropriate
    b[0        : k       ] += top
    b[ndim - k : ndim    ] += bottom
    b[0        : ndim : k] += left
    b[k-1      : ndim : k] += right
    
    return b 
make_b(2)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

: 

In [4]:
def radiator(k, width = .2, temperature = 100.):
    """Create one wall with a radiator
    Parameters: 
      k: number of grid points in each dimension; length of the wall.
      width: width of the radiator as a fraction of length of the wall (defaults to 0.2)
      temperature: temperature of the radiator (defaults to 100)
    Outputs:
      wall: the k element vector (as a numpy array) for the boundary conditions at the wall
    """
    rad_start = int(k * (0.5 - width/2))
    rad_end = int(k * (0.5 + width/2))
    wall = np.zeros(k)
    wall[rad_start : rad_end] = temperature
    
    return wall

## We will now set up the problem!

In [5]:
###########################
# Dimension of the square room (NOT the matrix)
# TRY using various values between 2 and 100
# k = 3    # Same as "simple" example given in lecture
k = 100

###########################
# Set up the width of the radiator
# Default (per the function def) is 0.2
# This is a number relative to the length of the side
rk = radiator(k, width = 1)
print('rk =', rk)

###########################
# Create the matrix!
# A will be the sparse nxn matrix (where n = k**2)
A = make_A(k)

###########################
# Create the b matrix: defines where the radiators go in the room
# SIDE=rk is placement of radiator, where SIDE can be (top, right, bottom, left)
b = make_b(k, right=rk)
#b = make_b(k, right=rk, left=rk, top=rk, bottom=rk)
print('b =', b)

###########################
# Print stuff to 's see what we got...
print("A: (sparse):\n",A)
print()
print("A (dense):\n", A.todense())
print("\nb:", b)
print("\nrad:", rk)

rk = [100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100.]
b = [  0.   0.   0. ...   0.   0. 100.]
A: (sparse):
   (0, 0)	4.0
  (0, 1)	-1.0
  (0, 100)	-1.0
  (1, 0)	-1.0
  (1, 1)	4.0
  (1, 2)	-1.0
  (1, 101)	-1.0
  (2, 1)	-1.0
  (2, 2)	4.0
  (2, 3)	-1.0
  (2, 102)	-1.0
  (3, 2)	-1.0
  (3, 3)	4.0
  (3, 4)	-1.0
  (3, 103)	-1.0
  (4, 3)	-1.0
  (4, 4)	4.0
  (4, 5)	-1.0
  (4, 104)	-1.0
  (5, 4)	-1.0
  (5, 5)	4.0
  (5, 6)	-1.0
  (5, 105)	-1.0
  (6, 5)	-1.0
  (6, 6)	4.0
  :	:
  (9993, 9993)	4.0
  (9993, 9994)	-1.0
  (9994, 9894)	-1.0
  (9994, 999

In [6]:
#print(npla.solve(A,b))
# It won't work... :(

# BECAUSE A is actually a SPARSE matrix (.solve() won't work with that format)

In [7]:
# This is how you go back to a dense matrix format:
AA = sparse.csr_matrix.todense(A)
print(AA)
#print(npla.solve(AA,b))

# But we won't use this format either...

[[ 4. -1.  0. ...  0.  0.  0.]
 [-1.  4. -1. ...  0.  0.  0.]
 [ 0. -1.  4. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  4. -1.  0.]
 [ 0.  0.  0. ... -1.  4. -1.]
 [ 0.  0.  0. ...  0. -1.  4.]]


In [8]:
# NOW!!! WE SOLVE IT!!
# We're going to use a different function (not .solve) altogether to solve Ax=b using a sparse matrix:
t = spla.spsolve(A,b)

# Reshape it back to a model (square) of the room
T = t.reshape(k,k)

print(t,"\n")
print(T)
# Looking at the data printed out is hopeless... you NEED to visualize it...

[1.07279116e-02 2.14666701e-02 3.22271367e-02 ... 2.09067518e+01
 3.02132781e+01 4.99892721e+01] 

[[1.07279116e-02 2.14666701e-02 3.22271367e-02 ... 2.09067518e+01
  3.02132781e+01 4.99892721e+01]
 [2.14449765e-02 4.29116319e-02 6.44216745e-02 ... 3.76070468e+01
  4.99570884e+01 6.97438103e+01]
 [3.21403623e-02 6.43132066e-02 9.65510583e-02 ... 4.99034489e+01
  6.22642183e+01 7.90288807e+01]
 ...
 [3.21403623e-02 6.43132066e-02 9.65510583e-02 ... 4.99034489e+01
  6.22642183e+01 7.90288807e+01]
 [2.14449765e-02 4.29116319e-02 6.44216745e-02 ... 3.76070468e+01
  4.99570884e+01 6.97438103e+01]
 [1.07279116e-02 2.14666701e-02 3.22271367e-02 ... 2.09067518e+01
  3.02132781e+01 4.99892721e+01]]


## Here come the Graphs!

**Let's start with a simple demonstration...**

In [9]:
# Simple demonstration of the plotting function in PyPlot

wave = np.sin( np.array(range(200)) / 100 * 2 * np.pi)

#print(wave)
plt.plot(wave)

### Ok... now for the real thing...

In [15]:
# Prep the plotter...
# Plot just a 2D image

#%matplotlib tk
plt.figure()
plt.imshow(T, cmap=cm.hot)

### Let's try and project the 2D graph onto a 3D representation!

In [11]:
print(k)

100


In [12]:
# Prep the plotter again...
X, Y = np.meshgrid(range(k), range(k))
#%matplotlib tk
#print(X,"\n")
#print(Y)

In [17]:
# Plot the 2D image as a 3D projection
fig = plt.figure()
ax = fig.gca()
ax = fig.gca('3d')
ax.plot_surface(X, Y, T, cmap=cm.hot)

AttributeError: 'Axes' object has no attribute 'plot_surface'